# Empirical Assignment (Q2)
## Economics 35310: Topics in Trade & Growth
### Author: Levi Crews

In this notebook I simulate an economy of 100,000 firms using the model from TKMD (2018). Firms are indexed from 1 to 100,000. Each set of eligible suppliers (max cardinality of 300) is drawn randomly from the set of firms whose indices precede that of the buyer. Each firm $j$ is characterized by
+ its set of eligible suppliers that satisfies the ordering ($Z_j$)
+ its core productivity level ($\phi_j$)
+ its vector of firm-pair-specific cost shifters ($\alpha_{kj}$)
+ its foreign input cost shifter ($\alpha_{Fj}$)
+ its foreign demand shifter ($\beta_{jF}$)
+ its vector of fixed cost draws for all eligible suppliers ($f_{kj}$)
+ its fixed costs of importing and exporting ($f_{Fj}$, $f_{jF}$)

I normalize firms' labor productivity shifters, $\alpha_{Lj} = 1$, and firms' domestic final demand shifters, $\beta_{jH} = 1$. I parameterize firm characteristics in the following way:
+ $\alpha_{kj} = X_k \times Y_{kj}$ with $X_k \sim $ Beta(0.1, 0.9) and $Y_{kj} \sim $ log-normal($\Phi_{scale}^{\alpha_{dom}}, \Phi_{disp}^{\alpha, \beta}$) i.i.d.
+ $\alpha_{Fj} \sim $ log-normal($\Phi_{scale}^{\alpha_F}, \Phi_{disp}^{\alpha, \beta}$) i.i.d.
+ $\beta_{jF} \sim $ log-normal($\Phi_{scale}^{\beta_F}, \Phi_{disp}^{\alpha, \beta}$) i.i.d.

For the 4 parameters $\Phi$ I use the estimated values in Table 4 of TKMD (2018):

| $\Phi_{scale}^{\alpha_{dom}}$ | $\Phi_{scale}^{\alpha_F}$ | $\Phi_{scale}^{\beta_F}$ | $\Phi_{disp}^{\alpha, \beta}$ |
|--|--|--|--|
| -4.42 | -2.22 | -2.01 | 2.34 |

**To simplify things greatly, I set all fixed costs of forming links to be the same.**

In this simulation I try to match the following moments of the Belgian data presented in Table 5 of TKMD (2018):
+ number of importing firms: 19,000
+ number of exporting firms: 12,000
+ labor share in cost: 0.17 / 0.34 / 0.54 (25/50/75 percentiles)
+ productivity: $\phi_j \sim $ log-normal (-1.52, 0.85)
+ total foreign input share: 0.24 / 0.39 / 0.55 (25/50/75 percentiles)
+ number of domestic suppliers: 19 / 33 / 55 (25/50/75 percentiles)
+ number of domestic customers: 2 / 9 / 34 (25/50/75 percentiles)

Note that I normalized $\frac{PE^{1/(\sigma-1)}}{\mu w} = 1$ to generate the productivity distribution. In addition, I calibrate $\sigma = 4$ and $\rho = 2$ following the baseline specification of TKMD (2018).

I will then (i) calculate the total foreign input share for each simulated firm and (ii) calculate the real wage change from going to autarky.

Finally, I will (iii) simulate a 10% increase in the import price and (iv) calculate the concomitant change in the real wage, holding the network fixed.

**My algorithm follows that described in Appendix G.1 of TKMD (2018).**

In [1]:
# Load packages 
using Distributions, StatsBase, Random, Statistics
using LinearAlgebra, SparseArrays, NLsolve, IterativeSolvers

In [2]:
# Load user-defined functions
# Change path as needed
include("Crews_Econ35310_FindW.jl")
include("Crews_Econ35310_FindA.jl")
include("Crews_Econ35310_Q2Solver.jl");

In [3]:
# Set seed
Random.seed!(4321);

### Initializing the Acyclic Network

Let's start by initializing an acyclic network. First, I index firms 1 to 100,000. Then, for each firm $j$ I draw the cardinality of its set of eligible domestic suppliers, $Z_j^D$ from a discretized log-normal(3.5, 0.8) distribution truncated at min$\{j,$ 300$\}$.

In [4]:
# Initialize network
num = 100000
maxcard = 300
Nmu = 3.5
Nsigma = 0.8

N = zeros(num, 2)
N[:, 1] = collect(1:num)
for i = 1:maxcard
    temp = min(rand(LogNormal(Nmu, Nsigma)), i-1)
    N[i, 2] = ceil(Int64, temp)
end
temp = min.(rand(LogNormal(Nmu, Nsigma), num-maxcard), maxcard*ones(num-maxcard,1))
N[maxcard+1:end, 2] = ceil.(Int64, temp)
N = Array{Int64}(N);

### Assigning Firm Characteristics

I now assign firm characteristics according to the distributions listed at the outset.

In [5]:
# Initialize parameters
Phi_ADscale = -4.42
Phi_AFscale = -2.22
Phi_BFscale = -2.01
Phi_ABdisp = 2.34
f = 2.0

# Draw firm characteristics
phi = rand(LogNormal(-1.52, 0.85), num) # \phi_j
X = rand(Beta(0.1, 0.9), num) # X_j
Y = rand(LogNormal(Phi_ADscale, Phi_ABdisp), num) # Y_{kj} for all k
alpha_H = X .* Y
alpha_F = rand(LogNormal(Phi_AFscale, Phi_ABdisp), num) # \alpha_{Fj}
beta_F = rand(LogNormal(Phi_BFscale, Phi_ABdisp), num) # \beta_{jF}
fixed = f*ones(num,1); # fixed cost

In [6]:
# Calibrate elasticities and labor supply
sigma = 4.0
rho = 2.0
tau = 1.05
mu = sigma / (sigma - 1)
L = 1.0;

# Arbitrarily choose foreign variable price vector
p_F = (rand(num,1) .+ 1.25) # p_F = 1.75 * ones(num,1)
P_F = 0.05 # P_F = (sum(p_F.^(1-sigma)))^(1/(sigma-1))
E_F = 0.7;

In [7]:
w = Crews_FindW(L, N, tau, p_F, P_F, E_F, sigma, rho, phi, alpha_H, alpha_F, beta_F, fixed)
if w >= 0
    println("w = ", w)
    println("Hooray! We found an equilibrium.")
else
    println("w = ", w)
    println("We failed to find an equilibrium.")
end

Aggregate demand blew up.


UndefVarError: UndefVarError: total_cost not defined

### The Effects of a 10% Foreign Price Increase

Consider a uniform 10% foreign price increase. We want to identify the concomitant change in the real wage. To do so, we used Propositions 1 and 2 as well as Section A.2 of the appendix of TKMD (2018).

In [ ]:
# 10% increase in foreign price
p_F_hat = 1.1

# guess w_hat
x_hat = ones(num,1)
x_hat_previous = 1.1*ones(num,1)
w_hat = 1    # guess
w_hat_previous = 0.9;

In [ ]:
while abs(w_hat_previous-w_hat) > 10e-9
    w_hat_previous = w_hat
    c_hat = ((ones(num,1)-totFshare)*(w_hat^(1-rho)) + totFshare*(p_F_hat^(1-rho))).^(1/(1-rho))
    x_hat_F = c_hat.^(1-sigma) .* IndI
    P_hat = (c_hat'.^(1-sigma)*share_H)^(1/(1-sigma))
    share_L_hat = w_hat^(1-rho) * c_hat.^(rho-1)
    share_hat = c_hat'.^(1-rho).*SparseNet.*c_hat.^(1-rho)
    while norm(x_hat-x_hat_previous) > 10e-30
        x_hat_previous = x_hat
        E_hat = (w*L/E*w_hat) + (total_profit[:,1]'*x_hat_previous[:,1]/E)
        x_hat = x_F./total_sale.*x_hat_F + x_H./total_sale.*c_hat.^(1-sigma)*P_hat^(sigma-1)*E_hat + share_hat.*x_D*x_hat_previous./total_sale
        w_hat = (share_L .* share_L_hat)'*(total_cost .* x_hat/(w*L))
        w_hat = w_hat[1]
    end
end

In [ ]:
real_wage_change = w_hat/P_hat